<a href="https://colab.research.google.com/github/jiwonlee-0218/colab/blob/main/RNN_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# 0. hyperparameter 상수화

#### hyperparamter 자주 바꾸게 되므로 python에는 상수라는 개념이 없지만 C처럼 맨 위에 상수처럼 선언하고 진행

#### (다른 kernel들을 보면 대문자로 맨 위에서 선언하고 진행하는 경우가 많습니다.)

max_features = 사용할 단어의 최대 개수

max_len = 문장의 최대 길이

embedding_dims = 단어 vector 내부의 원소 개수

In [ ]:
max_features = 20000 # 문장마다 단어의 갯수가 다른데 딥러닝에 넣을때는 각 단어의 갯수가 같아야 한다 그래서 그 단어를 맞춰주는 작업을 해야한다
# text data에서 단어를 vector로 바꿔줘야한다
# 하나의 데이터 즉 하나의 문장은 2d array
max_len = 200 # time_step
embedding_dims = 50 # feature

-----------

# 1. data upload

In [ ]:
from tensorflow.keras.datasets import imdb

## 1) data upload

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features) # imdb 데이터중 20000개를 사용하겠다는 소리

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

## 2) data에 대한 파악 - X

In [ ]:
x_train.shape

(25000,)

In [ ]:
x_train.ndim

1

In [ ]:
print('train data가 ' + str(len(x_train)) +'개')

train data가 25000개


NLP의 경우 text형태로 데이터가 들어있다

딥러닝 모델은 숫자를 받습니다.

text -> index(스칼라값) ->(embedding) -> vector

단어들을 숫자값으로 놓으면 의미관계들을 파악하기 어려우므로 vector로 바꿔주는데 이 작업을 embedding이라고한다

그럼 data 내부는 어떻게 구성되어 있는가?

원래는 text(문자 data) 자체를 vector로 변환하는 작업 필요하지만 NLP를 자세히 다루는 시간이 아니기 때문에 

숫자(vector) 형태의 data를 보게 됩니다.

In [ ]:
x_train # 25000개  이미 index로 바뀌어져있음  list 하나당 하나의 문장

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 82

25000개의 문장이 x_train에 담겨있다.

그럼 그 문장은 어떻게 생겼는가?

In [ ]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
len(x_train[0]) # 하나의 데이터인 한 문장 내에는 218개의 단어가 존재

218

In [ ]:
len(x_train[1])

189

각 문장의 길이가 다르다는 문제가 생김

## 3) data에 대한 파악 - y

In [ ]:
y_train.shape

(25000,)

In [ ]:
print('역시 train data가 ' + str(len(y_train)) +'개')

역시 train data가 25000개


그 값은 어떻게 구성이 되어있는가?

In [ ]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

0과 1이니 왠지 binary classfication? 

하지만 확인이 필요합니다.

In [ ]:
np.unique(y_train)

array([0, 1])

0과 1로만 구성이 되어있으므로 binary classification 맞습니다. (긍정 리뷰와 부정 리뷰 구분)

------

# 2. data preprocess

**오늘은 다른 전처리는 생략하고 진행하겠습니다.**

In [ ]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


아까 문장의 길이가 각각 다르므로 진행해야하는 것은 문장의 길이를 통일하는 방법

(이미지도 그렇고 전처리의 핵심 중 하나는 data의 형태를 동일하게 해주어야 합니다.)

--- 다시

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [ ]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 200)
x_test shape: (25000, 200)


In [ ]:
x_train[0].shape

(200,)

In [ ]:
x_train

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

------

### cf> input에 대한 이야기

현재 train_x.shape = (25000, 200)

현재 데이터셋 자체가 2d array이고 1개의 데이터는 1d array이다

우리가 RNN 모델에 넣어야 할 data의 ndim = 3 (25000, 200, input_dim)

즉, 각 단어를 vetor로 바꾸는 과정이 남았습니다.

[1,2,3,4,...,6,7..] 이 한개의 스칼라 값 자체를 vector로 바꿔줘야함

### 정리

#### <font color="blue"> 1> 한 문장당 단어 = max_len(200)개

(이 단어 각각을 또 vector로 만든다.) 

#### <font color="blue"> 2> 이 단어 각각 vector의 원소 개수 = embedding_dims(50)개

한 단어를 원소가 몇개인 vector로 만들것인가? 50개
    
(이 역할을 Embedding layer에서 진행) 즉 모델 내부에서 해결

# 3. build the model

simpleRNN 

- return_sequence = False => 2d array
- return_sequence = True => 3d array

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
# simpleRNN의 경우 return_sequences의 default가 False

In [ ]:
def create_model():
    model = Sequential()
    # 2d_array (data, max_len)
    model.add(Embedding(max_features, embedding_dims, input_length=max_len))
    # 3d_array (data, max_len, embedding_dims)
    model.add(SimpleRNN(32))
    # 2d_array (data, value)
    model.add(Dense(1, activation='sigmoid')) # Dense layer는 2d array 를 받는다
    # 1d_array [0, 1, 0, 0, 1]
    
    model.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'binary_crossentropy')

    return model

In [ ]:
model = create_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 50)           1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2656      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,002,689
Trainable params: 1,002,689
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train, validation_split=0.2, epochs=5, batch_size=512)

Epoch 1/5
40/40 [==============================] - 7s 154ms/step - loss: 0.6883 - accuracy: 0.5356 - val_loss: 0.6934 - val_accuracy: 0.5582
Epoch 2/5
40/40 [==============================] - 6s 144ms/step - loss: 0.5688 - accuracy: 0.7426 - val_loss: 0.4690 - val_accuracy: 0.8026
Epoch 3/5
40/40 [==============================] - 6s 143ms/step - loss: 0.3538 - accuracy: 0.8671 - val_loss: 0.3663 - val_accuracy: 0.8448
Epoch 4/5
40/40 [==============================] - 6s 144ms/step - loss: 0.2500 - accuracy: 0.9090 - val_loss: 0.3772 - val_accuracy: 0.8378
Epoch 5/5
40/40 [==============================] - 6s 144ms/step - loss: 0.1651 - accuracy: 0.9499 - val_loss: 0.3878 - val_accuracy: 0.8404


------

# 4. F1 score 사용

## 1) tensorflow의 metric을 사용

- tensorflow가 아직도 불안정한 면이 일부 있습니다. 잘 안 되는 분들은 2) 아래 예시로 하셔도 됩니다.

In [ ]:
# 설치가 필요한 분들은 아래 tensorflow_addons를 설치해주세요.

!pip install tensorflow_addons

     |████████████████████████████████| 706kB 7.6MB/s 


In [ ]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow_addons.metrics import F1Score

In [ ]:
precision = Precision()
recall = Recall()
f1score = F1Score(num_classes=1, threshold=0.5)

용어 그대로라면 class가 2개인 binary classification이니까 num_class = 2로 설정하는 게 맞습니다.

하지만 tensorflow 내부에서 num_classes를 2로 설정하면 output layer의 node를 2개로 간주하면서 model train할 때 tensor 간의 shape이 맞지 않는 문제가 생깁니다.

그래서 binary classification에서만 예외적으로 num_classes를 1로 설정하면 됩니다.

In [ ]:
def f1score_model():
    model = Sequential()
    # 2d_array (25000, 200)
    model.add(Embedding(max_features, embedding_dims, input_length=max_len))
    # 3d_array (25000, 200, 50)
    model.add(SimpleRNN(32, return_sequences = True)) # RNN의 output이 3d array가 됩니다.
    # 3d_array
    model.add(SimpleRNN(64)) # RNN모델을 여러층 쌓고 싶으면 return_sequences = True로 해줘야함
    # 2d_array
    model.add(Dense(1, activation='sigmoid'))
    # 1d_array

    
    model.compile(optimizer = 'adam', metrics=['accuracy', precision, recall, f1score], loss = 'binary_crossentropy')

    return model

In [ ]:
model_f = f1score_model()

In [ ]:
history = model_f.fit(x_train, y_train, validation_split=0.2, epochs=5, batch_size=512)

Epoch 1/5
40/40 [==============================] - 15s 328ms/step - loss: 0.6445 - accuracy: 0.5974 - precision: 0.6032 - recall: 0.6063 - f1_score: 0.6039 - val_loss: 0.3539 - val_accuracy: 0.8526 - val_precision: 0.8380 - val_recall: 0.8696 - val_f1_score: 0.8535
Epoch 2/5
40/40 [==============================] - 13s 313ms/step - loss: 0.2691 - accuracy: 0.8935 - precision: 0.8934 - recall: 0.8938 - f1_score: 0.8936 - val_loss: 0.3165 - val_accuracy: 0.8660 - val_precision: 0.8472 - val_recall: 0.8890 - val_f1_score: 0.8676
Epoch 3/5
40/40 [==============================] - 12s 312ms/step - loss: 0.1496 - accuracy: 0.9513 - precision: 0.9515 - recall: 0.9526 - f1_score: 0.9521 - val_loss: 0.3423 - val_accuracy: 0.8664 - val_precision: 0.8288 - val_recall: 0.9194 - val_f1_score: 0.8717
Epoch 4/5
40/40 [==============================] - 12s 312ms/step - loss: 0.0830 - accuracy: 0.9755 - precision: 0.9738 - recall: 0.9774 - f1_score: 0.9756 - val_loss: 0.4177 - val_accuracy: 0.8608 - va

precision과 recall의 조화평균(2ab/(a+b))를 구하면 f1_score가 나오는 것을 확인할 수 있습니다.

## 2) custom metrics

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def f1score_model():
    model = Sequential()
    # 2d_array (25000, 200)
    model.add(Embedding(max_features, embedding_dims, input_length=max_len))
    # 3d_array (25000, 200, 50)
    model.add(SimpleRNN(32, return_sequences = True)) # RNN의 output이 3d array가 됩니다.
    # 3d_array
    model.add(SimpleRNN(64))
    # 2d_array
    model.add(Dense(1, activation='sigmoid'))
    # 1d_array

    
    model.compile(optimizer = 'adam', metrics=['accuracy', recall_m, precision_m, f1_m], loss = 'binary_crossentropy')

    return model

In [ ]:
model_f = f1score_model()

In [ ]:
history = model_f.fit(x_train, y_train, validation_split=0.2, epochs=5, batch_size=512)

Epoch 1/5
40/40 [==============================] - 15s 328ms/step - loss: 0.6922 - accuracy: 0.5273 - recall_m: 0.5217 - precision_m: 0.5322 - f1_m: 0.5184 - val_loss: 0.5650 - val_accuracy: 0.7296 - val_recall_m: 0.6459 - val_precision_m: 0.7690 - val_f1_m: 0.7012
Epoch 2/5
40/40 [==============================] - 12s 310ms/step - loss: 0.5277 - accuracy: 0.7549 - recall_m: 0.7817 - precision_m: 0.7491 - f1_m: 0.7598 - val_loss: 0.4723 - val_accuracy: 0.7804 - val_recall_m: 0.8407 - val_precision_m: 0.7467 - val_f1_m: 0.7906
Epoch 3/5
40/40 [==============================] - 13s 314ms/step - loss: 0.2988 - accuracy: 0.8804 - recall_m: 0.8794 - precision_m: 0.8825 - f1_m: 0.8800 - val_loss: 0.4434 - val_accuracy: 0.8100 - val_recall_m: 0.8898 - val_precision_m: 0.7636 - val_f1_m: 0.8217
Epoch 4/5
40/40 [==============================] - 13s 315ms/step - loss: 0.1515 - accuracy: 0.9490 - recall_m: 0.9503 - precision_m: 0.9495 - f1_m: 0.9493 - val_loss: 0.4978 - val_accuracy: 0.7888 - va